In [117]:
import requests
import pandas as pd
import urllib.request

In [170]:
traffic_image_url='http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2'
headers_val={'AccountKey':'AO4qMbK3S7CWKSlplQZqlA=='}
traffic_image_req=requests.get(url=traffic_image_url,headers=headers_val)
traffic_image_df=pd.DataFrame(eval(traffic_image_req.content)['value'])

,CameraID,Latitude,Longitude,ImageLink
0,1001,1.295313,103.871146,https://dm-traffic-camera-itsc.s3.ap-southeast...
1,1002,1.319541,103.878563,https://dm-traffic-camera-itsc.s3.ap-southeast...
2,1003,1.323957,103.872858,https://dm-traffic-camera-itsc.s3.ap-southeast...
3,1004,1.319536,103.875067,https://dm-traffic-camera-itsc.s3.ap-southeast...
4,1005,1.363520,103.905394,https://dm-traffic-camera-itsc.s3.ap-southeast...
...,...,...,...,...
82,9702,1.394741,103.817971,https://dm-traffic-camera-itsc.s3.ap-southeast...
83,9703,1.422857,103.773005,https://dm-traffic-camera-itsc.s3.ap-southeast...
84,9704,1.422143,103.795421,https://dm-traffic-camera-itsc.s3.ap-southeast...
85,9705,1.426277,103.787166,https://dm-traffic-camera-itsc.s3.ap-southeast...


In [177]:
traffic_image_df['timestamp']=traffic_image_df['ImageLink'].apply(lambda x: x.split('/')[3] +' '+ x.split('/')[4].replace('-',''))
# .replace('-','/')
#

In [178]:
traffic_image_df['filename']=traffic_image_df.apply(lambda x:'%s_%s.jpg'%(x['CameraID'],x['timestamp']),axis=1)

In [180]:
traffic_image_df.apply(lambda x: urllib.request.urlretrieve(x['ImageLink'],x['filename']) ,axis=1)

0     (1001_2022-10-16 2150.jpg, [x-amz-id-2, x-amz-...
1     (1002_2022-10-16 2150.jpg, [x-amz-id-2, x-amz-...
2     (1003_2022-10-16 2150.jpg, [x-amz-id-2, x-amz-...
3     (1004_2022-10-16 2150.jpg, [x-amz-id-2, x-amz-...
4     (1005_2022-10-16 2150.jpg, [x-amz-id-2, x-amz-...
                            ...                        
82    (9702_2022-10-16 2150.jpg, [x-amz-id-2, x-amz-...
83    (9703_2022-10-16 2150.jpg, [x-amz-id-2, x-amz-...
84    (9704_2022-10-16 2150.jpg, [x-amz-id-2, x-amz-...
85    (9705_2022-10-16 2150.jpg, [x-amz-id-2, x-amz-...
86    (9706_2022-10-16 2150.jpg, [x-amz-id-2, x-amz-...
Length: 87, dtype: object

In [167]:
traffic_image_df['timestamp']=pd.to_datetime(traffic_image_df['timestamp'])
traffic_image_df

,CameraID,Latitude,Longitude,ImageLink,timestamp
0,1001,1.295313,103.871146,https://dm-traffic-camera-itsc.s3.ap-southeast...,2022-10-16 21:45:00
1,1002,1.319541,103.878563,https://dm-traffic-camera-itsc.s3.ap-southeast...,2022-10-16 21:45:00
2,1003,1.323957,103.872858,https://dm-traffic-camera-itsc.s3.ap-southeast...,2022-10-16 21:45:00
3,1004,1.319536,103.875067,https://dm-traffic-camera-itsc.s3.ap-southeast...,2022-10-16 21:45:00
4,1005,1.363520,103.905394,https://dm-traffic-camera-itsc.s3.ap-southeast...,2022-10-16 21:45:00
...,...,...,...,...,...
82,9702,1.394741,103.817971,https://dm-traffic-camera-itsc.s3.ap-southeast...,2022-10-16 21:45:00
83,9703,1.422857,103.773005,https://dm-traffic-camera-itsc.s3.ap-southeast...,2022-10-16 21:45:00
84,9704,1.422143,103.795421,https://dm-traffic-camera-itsc.s3.ap-southeast...,2022-10-16 21:45:00
85,9705,1.426277,103.787166,https://dm-traffic-camera-itsc.s3.ap-southeast...,2022-10-16 21:45:00


In [154]:
urllib.request.urlretrieve(traffic_image_df.loc[2,'ImageLink'],'test.jpg')


('test.jpg', <http.client.HTTPMessage at 0x2156be05130>)

In [181]:
traffic_speed_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficSpeedBandsv2'
traffic_speed_req=requests.get(url=traffic_speed_url,headers=headers_val)
traffic_speed_df=pd.DataFrame(eval(traffic_speed_req.content)['value'])
traffic_speed_df
# road category 
# A – Expressways
# B – Major Arterial Roads
# C – Arterial Roads
# D – Minor Arterial Roads
# E – Small Roads
# F – Slip Roads
# G – No category info available

# speed bands
# 1 – indicates speed range from 0 < 9
# 2 – indicates speed range from 10 < 19
# 3 – indicates speed range from 20 < 29
# 4 – indicates speed range from 30 < 39
# 5 – indicates speed range from 40 < 49
# 6 – indicates speed range from 50 < 59
# 7 – indicates speed range from 60 < 69
# 8 – speed range from 70 or more


,LinkID,RoadName,RoadCategory,SpeedBand,MinimumSpeed,MaximumSpeed,Location
0,103000000,KENT ROAD,E,2,10,19,1.3170142376560023 103.85298052044503 1.316684...
1,103000010,BUCKLEY ROAD,E,4,30,39,1.3166507852203482 103.84102305136321 1.316912...
2,103000011,BUCKLEY ROAD,E,3,20,29,1.316912438354752 103.84022564204443 1.3166507...
3,103000014,SHREWSBURY ROAD,E,5,40,49,1.3186726294030418 103.84700267615683 1.318021...
4,103000015,SHREWSBURY ROAD,E,5,40,49,1.3180212058893457 103.84711392225329 1.318672...
...,...,...,...,...,...,...,...
495,103001218,DRAYCOTT DRIVE,D,6,50,59,1.3096279339357806 103.83110989321554 1.309694...
496,103001219,DRAYCOTT DRIVE,D,4,30,39,1.309694857000047 103.83119447352243 1.3096279...
497,103001226,PADANG JERINGAU,D,2,10,19,1.3099051585628567 103.86554520014256 1.309253...
498,103001227,PADANG JERINGAU,D,8,70,999,1.309253800591554 103.86611733337693 1.3099051...


In [ ]:
traffic_speed_df[['SpeedBand','MinimumSpeed','MaximumSpeed']].drop_duplicates().sort_values('SpeedBand')

In [ ]:
traffic_incidents_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents'
traffic_incidents_req=requests.get(url=traffic_incidents_url,headers=headers_val)
traffic_incidents_df=pd.DataFrame(eval(traffic_incidents_req.content)['value'])
traffic_incidents_df

In [ ]:
traffic_incidents_df.loc[0,'Message']

In [ ]:
traffic_image_df.to_csv('traffic_image.csv',index=False)
traffic_speed_df.to_csv('traffic_speed.csv',index=False)
traffic_incidents_df.to_csv('traffic_incidents.csv',index=False)

In [88]:
weather_df=pd.read_csv('weather.csv')
weatherreq=requests.get(url='https://api.data.gov.sg/v1/environment/rainfall')

new_weather_df=pd.DataFrame(eval(weatherreq.content)['metadata']['stations'])

new_weather_df['latitude']=new_weather_df['location'].apply(lambda x: x['latitude'])
new_weather_df['longitude']=new_weather_df['location'].apply(lambda x: x['longitude'])
new_weather_df['timestamp']=eval(weatherreq.content)['items'][0]['timestamp']
new_weather_df['timestamp']=pd.to_datetime(new_weather_df['timestamp'])

station_rainfall=pd.DataFrame(eval(weatherreq.content)['items'][0]['readings']).rename(columns={'value':'rainfall in mm'})

new_weather_df=new_weather_df.merge(station_rainfall,how='left',left_on='id',right_on='station_id')
new_weather_df=new_weather_df.drop(['id','device_id','station_id','location'],axis=1)

pd.concat([new_weather_df,weather_df]).drop_duplicates().to_csv('weather.csv',index=False)

location name, no road name 
S201
S202
S203
S204
S205
S207
S208
S209
S210
S211
S212
S214
S216
S217
S219
S220
S222
S224
S226
S227
S228
S230


# Onemap (to get road name from lat long (WIP)

In [114]:
response=requests.post('https://developers.onemap.sg/privateapi/auth/post/getToken',json={'email':'leejin@u.nus.edu','password':'Whysohardtochange123!'})#.content

In [115]:
response

<Response [401]>

In [116]:
eval(response.content)

{'error': 'You are not an authorised user!'}

# END

In [ ]:
# id_options=[]
# for i in traffic_image_df['CameraID']:
#     temp={'label':'id%s'%str(i),'value':i}
#     id_options.append(temp)
# id_options